In [4]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer


from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_validate, KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier, ExtraTreesClassifier
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel

## Modelos 2

Uma continuação.

Agora inicialmente tentaremos verificar se a inclusão de MTRANS com um Colum transform e removendo 

In [3]:
df =pd.read_pickle('df_transformado.pkl')
df_test = pd.read_pickle('df_test_transformado.pkl')

df_o = pd.read_csv('../../playground-series-s4e2/train.csv')
df_test_o = pd.read_csv("../../playground-series-s4e2/test.csv")


### OneHot

Faremos o OneHot encoder nas colunas MTRANS do Dataframe original. Em seguida adicionaremos nos dataframes tratados

In [7]:
col_to_one_hot = ['MTRANS']
transformador = ColumnTransformer(
    transformers=[('', OneHotEncoder(), col_to_one_hot)],
    remainder='passthrough'  # mantém as colunas não transformadas
)

In [10]:
df_transformado = transformador.fit_transform(df_o)

# Convertendo a saída de volta para um DataFrame (opcional)
df_transformado = pd.DataFrame(df_transformado, columns=list(transformador.get_feature_names_out(df_o.columns)))
df_transformado.head()

,__MTRANS_Automobile,__MTRANS_Bike,__MTRANS_Motorbike,__MTRANS_Public_Transportation,__MTRANS_Walking,remainder__id,remainder__Gender,remainder__Age,remainder__Height,remainder__Weight,...,remainder__FCVC,remainder__NCP,remainder__CAEC,remainder__SMOKE,remainder__CH2O,remainder__SCC,remainder__FAF,remainder__TUE,remainder__CALC,remainder__NObeyesdad
0,0.0,0.0,0.0,1.0,0.0,0,Male,24.443011,1.699998,81.66995,...,2.0,2.983297,Sometimes,no,2.763573,no,0.0,0.976473,Sometimes,Overweight_Level_II
1,1.0,0.0,0.0,0.0,0.0,1,Female,18.0,1.56,57.0,...,2.0,3.0,Frequently,no,2.0,no,1.0,1.0,no,Normal_Weight
2,0.0,0.0,0.0,1.0,0.0,2,Female,18.0,1.71146,50.165754,...,1.880534,1.411685,Sometimes,no,1.910378,no,0.866045,1.673584,no,Insufficient_Weight
3,0.0,0.0,0.0,1.0,0.0,3,Female,20.952737,1.71073,131.274851,...,3.0,3.0,Sometimes,no,1.674061,no,1.467863,0.780199,Sometimes,Obesity_Type_III
4,0.0,0.0,0.0,1.0,0.0,4,Male,31.641081,1.914186,93.798055,...,2.679664,1.971472,Sometimes,no,1.979848,no,1.967973,0.931721,Sometimes,Overweight_Level_II


Como vismo anteriormente, praticamente todos os usuários usam apenas o Transporte Publico, a bicicleta e as próprias pernas. Então vamos incluir apenas essas colunas na base de dados. Repetiremos o processo para a base de teste.

In [11]:
df['AUTOMOBILE'] = df_transformado['__MTRANS_Automobile']
df['PUBLIC_TRANS'] = df_transformado['__MTRANS_Public_Transportation']
df['WALKING'] = df_transformado['__MTRANS_Walking']
df.head()

,__Gender_Female,__Gender_Male,id,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,...,SMOKE,CH2O,SCC,FAF,TUE,CALC,NObeyesdad,AUTOMOBILE,PUBLIC_TRANS,WALKING
0,0.0,1.0,0,24,1.699998,81.66995,1,1,2,3,...,0,2.763573,1,0.0,0.976473,1,3,0.0,1.0,0.0
1,1.0,0.0,1,18,1.56,57.0,1,1,2,3,...,0,2.0,1,1.0,1.0,0,1,1.0,0.0,0.0
2,1.0,0.0,2,18,1.71146,50.165754,1,1,2,1,...,0,1.910378,1,0.866045,1.673584,0,0,0.0,1.0,0.0
3,1.0,0.0,3,21,1.71073,131.274851,1,1,3,3,...,0,1.674061,1,1.467863,0.780199,1,6,0.0,1.0,0.0
4,0.0,1.0,4,32,1.914186,93.798055,1,1,3,2,...,0,1.979848,1,1.967973,0.931721,1,3,0.0,1.0,0.0


In [12]:
df_transformado = transformador.fit_transform(df_test_o)

# Convertendo a saída de volta para um DataFrame (opcional)
df_transformado = pd.DataFrame(df_transformado, columns=list(transformador.get_feature_names_out(df_test_o.columns)))
df_transformado.head()

,__MTRANS_Automobile,__MTRANS_Bike,__MTRANS_Motorbike,__MTRANS_Public_Transportation,__MTRANS_Walking,remainder__id,remainder__Gender,remainder__Age,remainder__Height,remainder__Weight,...,remainder__FAVC,remainder__FCVC,remainder__NCP,remainder__CAEC,remainder__SMOKE,remainder__CH2O,remainder__SCC,remainder__FAF,remainder__TUE,remainder__CALC
0,0.0,0.0,0.0,1.0,0.0,20758,Male,26.899886,1.848294,120.644178,...,yes,2.938616,3.0,Sometimes,no,2.825629,no,0.8554,0.0,Sometimes
1,0.0,0.0,0.0,1.0,0.0,20759,Female,21.0,1.6,66.0,...,yes,2.0,1.0,Sometimes,no,3.0,no,1.0,0.0,Sometimes
2,0.0,0.0,0.0,1.0,0.0,20760,Female,26.0,1.643355,111.600553,...,yes,3.0,3.0,Sometimes,no,2.621877,no,0.0,0.250502,Sometimes
3,0.0,0.0,0.0,1.0,0.0,20761,Male,20.979254,1.553127,103.669116,...,yes,2.0,2.977909,Sometimes,no,2.786417,no,0.094851,0.0,Sometimes
4,0.0,0.0,0.0,1.0,0.0,20762,Female,26.0,1.627396,104.835346,...,yes,3.0,3.0,Sometimes,no,2.653531,no,0.0,0.741069,Sometimes


In [13]:
df_test['AUTOMOBILE'] = df_transformado['__MTRANS_Automobile']
df_test['PUBLIC_TRANS'] = df_transformado['__MTRANS_Public_Transportation']
df_test['WALKING'] = df_transformado['__MTRANS_Walking']
df_test.head()

,__Gender_Female,__Gender_Male,id,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,AUTOMOBILE,PUBLIC_TRANS,WALKING
0,0.0,1.0,20758,27,1.848294,120.644178,1,1,3,3,1,0,2.825629,1,0.8554,0.0,1,0.0,1.0,0.0
1,1.0,0.0,20759,21,1.6,66.0,1,1,2,1,1,0,3.0,1,1.0,0.0,1,0.0,1.0,0.0
2,1.0,0.0,20760,26,1.643355,111.600553,1,1,3,3,1,0,2.621877,1,0.0,0.250502,1,0.0,1.0,0.0
3,0.0,1.0,20761,21,1.553127,103.669116,1,1,2,3,1,0,2.786417,1,0.094851,0.0,1,0.0,1.0,0.0
4,1.0,0.0,20762,26,1.627396,104.835346,1,1,3,3,1,0,2.653531,1,0.0,0.741069,1,0.0,1.0,0.0


### Modelo

KNN, Arvore de Decisão e o Gradiente Boosting

In [19]:
y = df['NObeyesdad']
id = df['id']
X = df.drop(['NObeyesdad','id'], axis=1)

y = y.to_numpy()
X = X.to_numpy()

id_test = df_test['id']
X_test = df_test.drop(['id'],axis=1).to_numpy()

In [21]:
nobsedy_invertio = {3: 'Overweight_Level_II', 1: 'Normal_Weight', 0: 'Insufficient_Weight',
                     6: 'Obesity_Type_III', 5: 'Obesity_Type_II', 2: 'Overweight_Level_I', 4: 'Obesity_Type_I'}


In [16]:
# k-fold cross validation
scores = list()
kfold = KFold(10, shuffle=True, random_state=4)

for k, (train, test) in enumerate(kfold.split(X, y)):
	# get data
	train_X, test_X = X[train], X[test]
	train_y, test_y = y[train], y[test]
	# fit model
	model = KNeighborsClassifier(n_neighbors=10)
	model.fit(train_X, train_y)
	# evaluate model
	ypred = model.predict(test_X)
	acc = accuracy_score(test_y, ypred)
	# store score
	scores.append(acc)
	print(f'fold {k}> accuracy = ', acc)

# summarize model performance
mean_s, std_s = np.mean(scores), np.std(scores)
print('Accuracy Mean: %.3f, Standard Deviation: %.3f' % (mean_s, std_s))


c:\Users\larae\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py:110: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\larae\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py", line 217, in _count_physical_cores
    raise ValueError(


fold 0> accuracy =  0.8578998073217726
fold 1> accuracy =  0.8526011560693642
fold 2> accuracy =  0.8473025048169557
fold 3> accuracy =  0.8521194605009634
fold 4> accuracy =  0.846820809248555
fold 5> accuracy =  0.8501926782273603
fold 6> accuracy =  0.861271676300578
fold 7> accuracy =  0.8492292870905588
fold 8> accuracy =  0.859277108433735
fold 9> accuracy =  0.8640963855421687
Accuracy Mean: 0.854, Standard Deviation: 0.006


In [17]:
# k-fold cross validation
scores = list()
kfold = KFold(10, shuffle=True, random_state=4)

for k, (train, test) in enumerate(kfold.split(X, y)):
	# get data
	train_X, test_X = X[train], X[test]
	train_y, test_y = y[train], y[test]
	# fit model
	model = RandomForestClassifier(n_estimators=300)
	model.fit(train_X, train_y)
	# evaluate model
	ypred = model.predict(test_X)
	acc = accuracy_score(test_y, ypred)
	# store score
	scores.append(acc)
	print(f'fold {k}> accuracy = ', acc)

# summarize model performance
mean_s, std_s = np.mean(scores), np.std(scores)
print('Accuracy Mean: %.3f, Standard Deviation: %.3f' % (mean_s, std_s))

fold 0> accuracy =  0.9060693641618497
fold 1> accuracy =  0.8978805394990366
fold 2> accuracy =  0.8882466281310212
fold 3> accuracy =  0.896917148362235
fold 4> accuracy =  0.8940269749518305
fold 5> accuracy =  0.8978805394990366
fold 6> accuracy =  0.9041425818882466
fold 7> accuracy =  0.8916184971098265
fold 8> accuracy =  0.9036144578313253
fold 9> accuracy =  0.8930120481927711
Accuracy Mean: 0.897, Standard Deviation: 0.006


In [18]:
scores = list()
kfold = KFold(10, shuffle=True, random_state=4)

for k, (train, test) in enumerate(kfold.split(X, y)):
	# get data
	train_X, test_X = X[train], X[test]
	train_y, test_y = y[train], y[test]
	# fit model
	model =  GradientBoostingClassifier()
	model.fit(train_X, train_y)
	# evaluate model
	ypred = model.predict(test_X)
	acc = accuracy_score(test_y, ypred)
	# store score
	scores.append(acc)
	print(f'fold {k}> accuracy = ', acc)

# summarize model performance
mean_s, std_s = np.mean(scores), np.std(scores)
print('Accuracy Mean: %.3f, Standard Deviation: %.3f' % (mean_s, std_s))

fold 0> accuracy =  0.9089595375722543
fold 1> accuracy =  0.9036608863198459
fold 2> accuracy =  0.8921001926782274
fold 3> accuracy =  0.8988439306358381
fold 4> accuracy =  0.901252408477842
fold 5> accuracy =  0.9051059730250481
fold 6> accuracy =  0.9065510597302505
fold 7> accuracy =  0.8940269749518305
fold 8> accuracy =  0.9127710843373494
fold 9> accuracy =  0.9065060240963856
Accuracy Mean: 0.903, Standard Deviation: 0.006


In [20]:
gb_model = GradientBoostingClassifier()
gb_model.fit(X,y)
y_pred = gb_model.predict(X_test)

In [22]:
df_gradient_boosting = pd.DataFrame({'id': id_test, 'NObeyesdad': y_pred})
df_gradient_boosting['NObeyesdad']  = df_gradient_boosting['NObeyesdad'].map(nobsedy_invertio)

df_gradient_boosting.to_csv('submtions/gradient_boosting2.csv', index=False, sep=',')

In [27]:
scores = list()
kfold = KFold(10, shuffle=True, random_state=4)

for k, (train, test) in enumerate(kfold.split(X, y)):
	# get data
	train_X, test_X = X[train], X[test]
	train_y, test_y = y[train], y[test]
	# fit model
	clf2 = RandomForestClassifier(n_estimators=300, random_state=1)
	clf1 = GradientBoostingClassifier()
	clf3 = GradientBoostingClassifier(n_estimators=300)
	model = VotingClassifier( estimators=[('gdb', clf1), ('rf', clf2), ('gdb300', clf3)],  voting='hard')
	model.fit(train_X, train_y)
	# evaluate model
	ypred = model.predict(test_X)
	acc = accuracy_score(test_y, ypred)
	# store score
	scores.append(acc)
	print(f'fold {k}> accuracy = ', acc)

# summarize model performance
mean_s, std_s = np.mean(scores), np.std(scores)
print('Accuracy Mean: %.3f, Standard Deviation: %.3f' % (mean_s, std_s))

fold 0> accuracy =  0.9128131021194605
fold 1> accuracy =  0.9070327552986512
fold 2> accuracy =  0.8959537572254336
fold 3> accuracy =  0.9007707129094412
fold 4> accuracy =  0.9065510597302505
fold 5> accuracy =  0.9094412331406551
fold 6> accuracy =  0.9084778420038536
fold 7> accuracy =  0.8973988439306358
fold 8> accuracy =  0.9185542168674699
fold 9> accuracy =  0.9132530120481928
Accuracy Mean: 0.907, Standard Deviation: 0.007


In [25]:
clf2 = RandomForestClassifier(n_estimators=300, random_state=1)
clf1 = GradientBoostingClassifier()
clf3 = GradientBoostingClassifier(n_estimators=300)
model = VotingClassifier( estimators=[('gdb', clf1), ('rf', clf2), ('gdb300', clf3)],  voting='hard')
model.fit(X, y)
y_pred = model.predict(X_test)

In [26]:
df_voting = pd.DataFrame({'id': id_test, 'NObeyesdad': y_pred})
df_voting['NObeyesdad']  = df_voting['NObeyesdad'].map(nobsedy_invertio)

df_voting.to_csv('submtions/voting3.csv', index=False, sep=',')

Considerando os resultado do comitê de votação com todos os dados, e conseguindo assim o score de 0.90245 no Kaggle.

Iremos agora modificar umas colunas, visto que tentamos remover o TUE, SMOKE e não deu muito certo. Vamos adicionar o IMC

In [28]:
df['IMC'] = df['Weight'] / (df['Height']**2)
df_test['IMC'] = df_test['Weight'] / (df['Height']**2)
df.head()

,__Gender_Female,__Gender_Male,id,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,...,CH2O,SCC,FAF,TUE,CALC,NObeyesdad,AUTOMOBILE,PUBLIC_TRANS,WALKING,IMC
0,0.0,1.0,0,24,1.699998,81.66995,1,1,2,3,...,2.763573,1,0.0,0.976473,1,3,0.0,1.0,0.0,28.259565
1,1.0,0.0,1,18,1.56,57.0,1,1,2,3,...,2.0,1,1.0,1.0,0,1,1.0,0.0,0.0,23.422091
2,1.0,0.0,2,18,1.71146,50.165754,1,1,2,1,...,1.910378,1,0.866045,1.673584,0,0,0.0,1.0,0.0,17.126706
3,1.0,0.0,3,21,1.71073,131.274851,1,1,3,3,...,1.674061,1,1.467863,0.780199,1,6,0.0,1.0,0.0,44.855798
4,0.0,1.0,4,32,1.914186,93.798055,1,1,3,2,...,1.979848,1,1.967973,0.931721,1,3,0.0,1.0,0.0,25.599151


In [30]:
y = df['NObeyesdad']
id = df['id']
X = df.drop(['NObeyesdad','id'], axis=1)

y = y.to_numpy()
X = X.to_numpy()

id_test = df_test['id']
X_test = df_test.drop(['id'],axis=1).to_numpy()

In [32]:
# k-fold cross validation
scores = list()
kfold = KFold(10, shuffle=True, random_state=4)

for k, (train, test) in enumerate(kfold.split(X, y)):
	# get data
	train_X, test_X = X[train], X[test]
	train_y, test_y = y[train], y[test]
	# fit model
	model = RandomForestClassifier(n_estimators=300)
	model.fit(train_X, train_y)
	# evaluate model
	ypred = model.predict(test_X)
	acc = accuracy_score(test_y, ypred)
	# store score
	scores.append(acc)
	print(f'fold {k}> accuracy = ', acc)

# summarize model performance
mean_s, std_s = np.mean(scores), np.std(scores)
print('Accuracy Mean: %.3f, Standard Deviation: %.3f' % (mean_s, std_s))

fold 0> accuracy =  0.9108863198458574
fold 1> accuracy =  0.8988439306358381
fold 2> accuracy =  0.890655105973025
fold 3> accuracy =  0.8959537572254336
fold 4> accuracy =  0.9017341040462428
fold 5> accuracy =  0.9022157996146436
fold 6> accuracy =  0.903179190751445
fold 7> accuracy =  0.8959537572254336
fold 8> accuracy =  0.908433734939759
fold 9> accuracy =  0.9079518072289157
Accuracy Mean: 0.902, Standard Deviation: 0.006


Observamos que houve um aumento significativo nos testes em relação a acurácia média do KNN, todavia uma estabilidade média nos algoritmos do Random Forest e Gradient Boost. No geral diminuiu a precisão deles.

In [33]:
# k-fold cross validation
scores = list()
kfold = KFold(10, shuffle=True, random_state=4)

for k, (train, test) in enumerate(kfold.split(X, y)):
	# get data
	train_X, test_X = X[train], X[test]
	train_y, test_y = y[train], y[test]
	# fit model
	model = RandomForestClassifier(n_estimators=300)
	model.fit(train_X, train_y)
	# evaluate model
	ypred = model.predict(test_X)
	acc = accuracy_score(test_y, ypred)
	# store score
	scores.append(acc)
	print(f'fold {k}> accuracy = ', acc)

# summarize model performance
mean_s, std_s = np.mean(scores), np.std(scores)
print('Accuracy Mean: %.3f, Standard Deviation: %.3f' % (mean_s, std_s))

fold 0> accuracy =  0.9099229287090559
fold 1> accuracy =  0.8978805394990366
fold 2> accuracy =  0.8882466281310212
fold 3> accuracy =  0.8973988439306358
fold 4> accuracy =  0.9002890173410405
fold 5> accuracy =  0.9002890173410405
fold 6> accuracy =  0.8988439306358381
fold 7> accuracy =  0.896917148362235
fold 8> accuracy =  0.9055421686746988
fold 9> accuracy =  0.9074698795180723
Accuracy Mean: 0.900, Standard Deviation: 0.006


In [34]:
scores = list()
kfold = KFold(10, shuffle=True, random_state=4)

for k, (train, test) in enumerate(kfold.split(X, y)):
	# get data
	train_X, test_X = X[train], X[test]
	train_y, test_y = y[train], y[test]
	# fit model
	model =  GradientBoostingClassifier()
	model.fit(train_X, train_y)
	# evaluate model
	ypred = model.predict(test_X)
	acc = accuracy_score(test_y, ypred)
	# store score
	scores.append(acc)
	print(f'fold {k}> accuracy = ', acc)

# summarize model performance
mean_s, std_s = np.mean(scores), np.std(scores)
print('Accuracy Mean: %.3f, Standard Deviation: %.3f' % (mean_s, std_s))

fold 0> accuracy =  0.9070327552986512
fold 1> accuracy =  0.8973988439306358
fold 2> accuracy =  0.8877649325626205
fold 3> accuracy =  0.9041425818882466
fold 4> accuracy =  0.9051059730250481
fold 5> accuracy =  0.8964354527938343
fold 6> accuracy =  0.903179190751445
fold 7> accuracy =  0.8930635838150289
fold 8> accuracy =  0.9050602409638554
fold 9> accuracy =  0.9026506024096386
Accuracy Mean: 0.900, Standard Deviation: 0.006


In [35]:
scores = list()
kfold = KFold(10, shuffle=True, random_state=4)

for k, (train, test) in enumerate(kfold.split(X, y)):
	# get data
	train_X, test_X = X[train], X[test]
	train_y, test_y = y[train], y[test]
	# fit model
	model =  GradientBoostingClassifier()
	model.fit(train_X, train_y)
	# evaluate model
	ypred = model.predict(test_X)
	acc = accuracy_score(test_y, ypred)
	# store score
	scores.append(acc)
	print(f'fold {k}> accuracy = ', acc)

# summarize model performance
mean_s, std_s = np.mean(scores), np.std(scores)
print('Accuracy Mean: %.3f, Standard Deviation: %.3f' % (mean_s, std_s))

fold 0> accuracy =  0.9075144508670521
fold 1> accuracy =  0.8973988439306358
fold 2> accuracy =  0.8877649325626205
fold 3> accuracy =  0.9041425818882466
fold 4> accuracy =  0.9051059730250481
fold 5> accuracy =  0.8964354527938343
fold 6> accuracy =  0.903179190751445
fold 7> accuracy =  0.8930635838150289
fold 8> accuracy =  0.9050602409638554
fold 9> accuracy =  0.9026506024096386
Accuracy Mean: 0.900, Standard Deviation: 0.006


In [ ]:
df.to_pickle('df_after_models.pkl')
df_test.to_pickle('df_test_after_models.pkl')

Vamos reanalizar os dados, recortar alguns. Embora TUE e SMOKE tenham sido indicados como parametros que pouco afetam as árvores, mas ao retirar diminuiu a precisão da árvore e gradient boost